In [1]:
import pandas as pd
import py4cytoscape as p4c
import glob

In [2]:
p4c.cytoscape_ping()
p4c.cytoscape_version_info()

You are connected to Cytoscape!


{'apiVersion': 'v1',
 'cytoscapeVersion': '3.8.2',
 'automationAPIVersion': '1.2.0',
 'py4cytoscapeVersion': '0.0.9'}

## Test con bladder

In [23]:
p4c.sandbox_send_to("../regulaciontrans-data-server/bladder/network_aracne/cancer-interactions-10000.tsv", "bladder_cancer.tsv")

{'filePath': '/home/diana/CytoscapeConfiguration/filetransfer/default_sandbox/bladder_cancer.tsv'}

In [24]:
net = p4c.import_network_from_tabular_file("bladder_cancer.tsv", first_row_as_column_names=True,
                                     column_type_list = "s,t,ea,ea,ea,ea,ea") 

In [25]:
vertices = pd.read_csv("../regulaciontrans-data-server/bladder/network_aracne/cancer-vertices-10000.tsv", sep="\t")

In [26]:
vertices.head()

,ensembl,chr,start,end,symbol,cytoband
0,ENSG00000000460,1,169662007,169854080,C1orf112,q24.2
1,ENSG00000001631,7,92198969,92246166,KRIT1,q21.2
2,ENSG00000004534,3,49940007,50100045,RBM6,p21.31
3,ENSG00000005100,17,5440912,5469060,DHX33,p13.2
4,ENSG00000006744,17,12992391,13018187,ELAC2,p12


In [27]:
vertices.rename(columns={'ensembl': 'name'}, inplace=True)

In [28]:
p4c.load_table_data(vertices, data_key_column="name", network = net["networks"][0])

'Success: Data loaded in defaultnode table'

In [29]:
node_attrs = pd.read_csv("../regulaciontrans-data-server/bladder/network_aracne/cancer-node-attributes-10000.tsv", sep="\t")

In [30]:
p4c.load_table_data(node_attrs, data_key_column="name", network = net["networks"][0])

'Success: Data loaded in defaultnode table'

In [31]:
deg = pd.read_csv(glob.glob("../regulaciontrans-data-server/bladder/deg/*_*_*_si-arsyn_deg_results.tsv")[0], sep="\t")

In [32]:
deg.rename(columns={'ensembl_id': 'name'}, inplace=True)

In [33]:
p4c.load_table_data(deg, data_key_column="name", network = net["networks"][0])

/home/diana/.local/lib/python3.9/site-packages/py4cytoscape/tables.py:474: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset[col] = col_val


'Success: Data loaded in defaultnode table'

In [34]:
interactions = pd.read_csv("../regulaciontrans-data-server/bladder/network_aracne/cancer-interactions-10000.tsv", 
                          sep = "\t")

In [35]:
interactions.head()

,source_ensembl,target_ensembl,mi,distance,row_num,interaction_type,cond
0,ENSG00000197903,ENSG00000276180,0.748817,7830,1,Intra,cancer
1,ENSG00000187583,ENSG00000187642,0.723436,8707,2,Intra,cancer
2,ENSG00000159189,ENSG00000173369,0.665461,9132,3,Intra,cancer
3,ENSG00000196866,ENSG00000197409,0.643737,2011,4,Intra,cancer
4,ENSG00000188536,ENSG00000244734,0.614236,-1,5,Inter,cancer


This way we can select a new collection for each network, to avoid setting the same deg attributes to a node in all networks

In [38]:
p4c.networks.create_network_from_data_frames(nodes = vertices,  edges = interactions, 
                                             node_id_list  = "name", 
                                             source_id_list = "source_ensembl", 
                                             target_id_list = "target_ensembl",
                                             interaction_type_list = "interaction_type",
                                            collection = "bladder", title="bladder")

Applying default style...
Applying preferred layout


{'networkSUID': 56759}

## Todos los tejidos

In [5]:
tissues = ["bladder", "brain", "breast", "colorectal", "esophagus",
            "kidney", "liver", "lung", "ovary", "pancreas", "prostate",
            "testis", "thyroid","skin", "uterus"]

## Cancer

In [4]:
def loadCancerNetworks(ts, ni, newNet):
    for t in ts:
        print("Working with tissue: " + t)
        if newNet:
            print("Reading interactions\n")
            interactions = pd.read_csv("../regulaciontrans-data-server/" + t 
                                       +"/network_aracne/cancer-interactions-" + ni + ".tsv",
                                       sep = "\t") 
            print("Reading vertices\n")
            vertices = pd.read_csv("../regulaciontrans-data-server/"+t+"/network_aracne/cancer-vertices-"+ni+".tsv", sep="\t")
            vertices.rename(columns={'ensembl': 'name'}, inplace=True)
            print("Loading network\n")
            net = p4c.networks.create_network_from_data_frames(nodes = vertices,  edges = interactions, 
                                                 node_id_list  = "name", 
                                                 source_id_list = "source_ensembl", 
                                                 target_id_list = "target_ensembl",
                                                 interaction_type_list = "interaction_type",
                                                collection = t,  title = t + "_cancer")
        else:
            suid = p4c.networks.get_network_suid(title = t + "_cancer")
            print("Got network SUID" + str(suid))
            net = {"networkSUID": suid}
        print("Setting node attributes\n")
        #node_attrs = pd.read_csv("../regulaciontrans-data-server/"+t+"/network_aracne/cancer-node-attributes-"+ni+".tsv", sep="\t")
        #p4c.load_table_data(node_attrs, data_key_column="name", network = net["networkSUID"])
        print("Setting deg info\n")
        deg = pd.read_csv(glob.glob("../regulaciontrans-data-server/"+t+"/deg/*_*_*_si-arsyn_deg_results.tsv")[0], sep="\t")
        deg.rename(columns={'ensembl_id': 'name'}, inplace=True)
        p4c.load_table_data(deg, data_key_column="name", network = net["networkSUID"])
        #print("Setting node communities\n")
        #comms = pd.read_csv("../regulaciontrans-data-server/"+t+"/network_aracne/communities/cancer-comm-all-louvain"+ni+".tsv", sep="\t")
        #comms.rename(columns={'ensembl': 'name'}, inplace=True)
        #p4c.load_table_data(comms, data_key_column="name", network = net["networkSUID"])
    

In [7]:
loadCancerNetworks(tissues, "10000", False)

Working with tissue: bladder
Got network SUID769309
Setting node attributes

Setting deg info



/home/diana/.local/lib/python3.9/site-packages/py4cytoscape/tables.py:474: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset[col] = col_val


Working with tissue: brain
Got network SUID1424957
Setting node attributes

Setting deg info



/home/diana/.local/lib/python3.9/site-packages/py4cytoscape/tables.py:474: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset[col] = col_val


Working with tissue: breast
Got network SUID1205071
Setting node attributes

Setting deg info



...

/home/diana/.local/lib/python3.9/site-packages/py4cytoscape/tables.py:474: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset[col] = col_val


Working with tissue: skin
Got network SUID1095194
Setting node attributes

Setting deg info



/home/diana/.local/lib/python3.9/site-packages/py4cytoscape/tables.py:474: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset[col] = col_val


Working with tissue: uterus
Got network SUID219871
Setting node attributes

Setting deg info



/home/diana/.local/lib/python3.9/site-packages/py4cytoscape/tables.py:474: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset[col] = col_val


In [4]:
def loadNormalNetworks(ts, ni, newNet):
    for t in ts:
        print("Working with tissue: " + t)
        if newNet:
            print("Reading interactions\n")
            interactions = pd.read_csv("../regulaciontrans-data-server/" + t 
                                   +"/network_aracne/normal-interactions-" + ni + ".tsv",
                                   sep = "\t") 
            print("Reading vertices\n")
            vertices = pd.read_csv("../regulaciontrans-data-server/"+t+"/network_aracne/normal-vertices-"+ni+".tsv", sep="\t")
            vertices.rename(columns={'ensembl': 'name'}, inplace=True)
            print("Loading network\n")
            net = p4c.networks.create_network_from_data_frames(nodes = vertices,  edges = interactions, 
                                                 node_id_list  = "name", 
                                             source_id_list = "source_ensembl", 
                                             target_id_list = "target_ensembl",
                                             interaction_type_list = "interaction_type",
                                            collection = t,  title = t + "_normal")
        
        else:
            suid = p4c.networks.get_network_suid(title = t + "_normal")
            print("Got network SUID" + str(suid))
            net = {"networkSUID": suid}
            print("Setting node attributes\n")
            node_attrs = pd.read_csv("../regulaciontrans-data-server/"+t+"/network_aracne/normal-node-attributes-"+ni+".tsv", sep="\t")
            p4c.load_table_data(node_attrs, data_key_column="name", network = net["networkSUID"])
            comms = pd.read_csv("../regulaciontrans-data-server/"+t+"/network_aracne/communities/normal-comm-all-"+ni+".tsv", sep="\t")
            comms.rename(columns={'ensembl': 'name'}, inplace=True)
            p4c.load_table_data(comms, data_key_column="name", network = net["networkSUID"])

In [7]:
loadNormalNetworks(tissues, "10000")

Applying default style...
Applying preferred layout
Applying default style...
Applying preferred layout
Applying default style...
Applying preferred layout
Applying default style...
Applying preferred layout
Applying default style...
Applying preferred layout
Applying default style...
Applying preferred layout
Applying default style...
Applying preferred layout
Applying default style...
Applying preferred layout
Applying default style...
Applying preferred layout
Applying default style...
Applying preferred layout
Applying default style...
Applying preferred layout
Applying default style...
Applying preferred layout
Applying default style...
Applying preferred layout
Applying default style...
Applying preferred layout
Applying default style...
Applying preferred layout


### The viewTreshold value in Cytoscape must be changed in cytoscape to load 100k networks.

In [14]:
loadCancerNetworks(tissues, "100000", False)

Working with tissue: bladder
Got network SUID763521
Setting node attributes

Setting deg info



/home/diana/.local/lib/python3.9/site-packages/py4cytoscape/tables.py:474: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset[col] = col_val


Setting node communities

Working with tissue: brain
Got network SUID654757
Setting node attributes

Setting deg info



/home/diana/.local/lib/python3.9/site-packages/py4cytoscape/tables.py:474: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset[col] = col_val


Setting node communities

Working with tissue: breast
Got network SUID1312797
Setting node attributes

Setting deg info



...

Setting node communities

Working with tissue: skin
Got network SUID872950
Setting node attributes

Setting deg info



/home/diana/.local/lib/python3.9/site-packages/py4cytoscape/tables.py:474: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset[col] = col_val


Setting node communities

Working with tissue: uterus
Got network SUID18
Setting node attributes

Setting deg info



/home/diana/.local/lib/python3.9/site-packages/py4cytoscape/tables.py:474: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset[col] = col_val


Setting node communities



In [37]:
loadNormalNetworks(tissues, "100000", False)

Working with tissue: bladder
Got network SUID51
Setting node attributes

Working with tissue: brain
Got network SUID226520
Setting node attributes

Working with tissue: breast
Got network SUID444341
Setting node attributes

Working with tissue: colorectal
Got network SUID668172
Setting node attributes

Working with tissue: esophagus
Got network SUID893689
Setting node attributes

Working with tissue: kidney
Got network SUID1110366
Setting node attributes

Working with tissue: liver
Got network SUID1332303
Setting node attributes

Working with tissue: lung
Got network SUID1557166
Setting node attributes

Working with tissue: ovary
Got network SUID1782131
Setting node attributes

Working with tissue: pancreas
Got network SUID2216441
Setting node attributes

Working with tissue: prostate
Got network SUID2432582
Setting node attributes

Working with tissue: testis
Got network SUID2651837
Setting node attributes

Working with tissue: thyroid
Got network SUID2865514
Setting node attributes



In [3]:
def loadEdgesInfo(ts, ni):
    for t in ts:
        print("Working with tissue: " + t)
        print("Reading interactions\n")
        interactions = pd.read_csv("../regulaciontrans-data-server/" + t 
                                   +"/network_aracne/cancer-interactions-" + ni + ".tsv",
                                   sep = "\t")
        suid = p4c.networks.get_network_suid(title = t + "_cancer")
        print("Got network SUID" + str(suid))
        net = {"networkSUID": suid}
        print("Building edges names")
        interactions["name"] = interactions["source_ensembl"] + " (" + interactions["interaction_type"] + ") " + interactions["target_ensembl"]
        interactions = interactions[["name", "mi", "row_num", "distance"]]
        print("Adding edges attributes")
        p4c.load_table_data(interactions, data_key_column="name", network = net["networkSUID"], table = "edge")
       

In [4]:
tissues = ["bladder", "brain", "breast", "colorectal", "esophagus",
            "kidney", "liver", "lung", "ovary", "pancreas", "prostate",
            "testis", "thyroid","skin", "uterus"]

In [10]:
loadEdgesInfo([tissues[14]], "100000")

Working with tissue: uterus
Reading interactions

Got network SUID1532902
Building edges names
Adding edges attributes
